# Word embeddings tutorial

https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html

Words can be represented as on-hot-encodings. However there are drawbacks to this representation as the words are treated as independent entities with no relation to each other. Also this requires a huge amount memory.

We need a representation that has some notion of similarity between words

Imagine we have the following three sentences

* The mathematician ran to the store
* The physicist ran to the store
* The mathematician solved the open problem

Now suppose we get a new sentence never seen before in our data set.

* The physicist solved the open problem

Our language model should recognize

1. We have seen mathematician and physicist in the sam erole in a sentence
2. We have seen mathematician in the same role as physicist in this new unseen sentence.

This example relies on a fundamental linguistic assumption that words appearing in similar contexts are related to each other.

This is called the [distributional hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics)

Instead of the sparse one-hot vector representation we use a dense (values typically non-zero) vector representation for mathematician $v_m$ and physicist $v_p$

We can find the similarity between the two values by using the consine distance.

$$ cos(\phi) = \frac{ v_m \cdot v_p }{ | v_m | \cdot | v_p |} $$

We let the word embeddings be parameters in our neural network model and the model will learn the representations during training.

Word embeddings are an efficiently encoded semantic representation of a word.

## Word embeddings in Pytorch

Pytorch stores embeddings where each word is a unique index into a lookup table. The embeddings are stored as a $|V| \cdot D$ matrix where $V$ is the vocabulary and $D$ is the dimensionality of the embeddings.

### Import libraries

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
import matplotlib.pyplot as plt

In [3]:
torch.manual_seed(1)

### Example embedding layer

In [4]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


## Example N-Gram language modelling

#### Get data

In [5]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

Get tuples of data

([word - CONTEXT_SIZE, ... , word - 1], target_word)

In [6]:
ngrams = [
    ([test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)], test_sentence[i])
    for i in range(CONTEXT_SIZE, len(test_sentence))
]

In [7]:
print(ngrams[:3])

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


In [8]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

Define neural network model

In [9]:
class NGramLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 120)
        self.linear2 = nn.Linear(120, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        logs_probs = F.log_softmax(out, dim=1)
        return logs_probs

Create model and set up optimizer

In [10]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModel(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.01)
model

NGramLanguageModel(
  (embeddings): Embedding(97, 10)
  (linear1): Linear(in_features=20, out_features=120, bias=True)
  (linear2): Linear(in_features=120, out_features=97, bias=True)
)

Train model

In [11]:
for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:
        context_idxs = torch.tensor(
            [word_to_ix[w] for w in context], dtype=torch.long
        )
        model.zero_grad()
        log_probs = model(context_idxs)
        loss = loss_function(
            log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long)
        )
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss)

print('\n'.join(f'{loss:.1f}' for loss in losses[-10:]))

523.1
500.3
478.7
457.5
436.2
414.5
392.1
368.9
344.8
319.9


Show embeddings for a single word

In [12]:
print(model.embeddings.weight[word_to_ix['beauty']])

tensor([ 0.6702,  0.4168, -1.0248,  0.9977,  0.8020, -2.2386, -1.5171, -0.9094,
         0.8163, -0.6731], grad_fn=<SelectBackward0>)


## Computing Word Embeddings: Continuous Bag-of-Words

The continuous bag-of-words model (CBOW) predicts words given the context of a few words before and a few words after the target word. CBOW is not sequential and does not have to be probabilistic but is typically used to train word embeddings.

Given a target word $w_i$ and an N context window on each side, $w_{i - 1}, ..., w_{i - N}$ and $w_{i + 1}, ..., w_{i + N}$ called the context words $C$, CBOW trieds to minimize the following.

$$ -\log p(w_i|C) = -\log \textrm{softmax} \big ( A(\sum \limits_{w \in C} {q_{w}} ) + b \big ) $$

In [13]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 words to the right
raw_text = """"We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

In [14]:
vocab = set(raw_text)
vocab_size = len(vocab)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [15]:
cbow_data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = (
        [raw_text[i - j - 1] for j in range(CONTEXT_SIZE)] +
        [raw_text[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = raw_text[i]
    cbow_data.append((context, target))
print(cbow_data[:5])        

[(['are', '"We', 'to', 'study'], 'about'), (['about', 'are', 'study', 'the'], 'to'), (['to', 'about', 'the', 'idea'], 'study'), (['study', 'to', 'idea', 'of'], 'the'), (['the', 'study', 'of', 'a'], 'idea')]


Define model

In [16]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(2 * context_size * embedding_dim, 120)
        self.linear2 = nn.Linear(120, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        logs_probs = F.log_softmax(out, dim=1)
        return logs_probs

Create model, loss function and optimizer

In [17]:
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.01)
model

CBOW(
  (embeddings): Embedding(49, 10)
  (linear1): Linear(in_features=40, out_features=120, bias=True)
  (linear2): Linear(in_features=120, out_features=49, bias=True)
)

In [18]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

def make_target_vector(target, word_to_ix):
    idx = word_to_ix[target]
    return torch.tensor([idx], dtype=torch.long)

In [19]:
for epoch in range(10):
    total_loss = 0
    for context, target in cbow_data:
        context_idxs = make_context_vector(context, word_to_ix)
        model.zero_grad()
        log_probs = model(context_idxs)
        loss = loss_function(
            log_probs, make_target_vector(target, word_to_ix)
        )
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss)

print('\n'.join(f'{loss:.1f}' for loss in losses[-10:]))

232.3
218.7
205.8
193.1
180.4
167.6
154.6
141.5
128.2
114.9
